In [1]:
import sys
from pyspark import SparkContext
import json

In [2]:
# task1, input format: task1.py <review_filepath> <output_filepath>
# review_path = sys.argv[0]
# output_path = sys.argv[1]
review_path = "./test_review.json"
output_path = "./ans1.json"
sc = SparkContext("local[*]",appName="task1").getOrCreate()
review = sc.textFile(review_path).map(lambda x: json.loads(x))
output = {}

23/02/03 22:40:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/03 22:40:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# A. The total number of reviews
output["n_review"] = review.map(lambda x: x["review_id"]).distinct().count()

In [4]:
# B. The number of reviews in 2018
output["n_review_2018"] = review.filter(lambda x: x["date"].startswith("2018")).\
                                map(lambda x: x["review_id"]).distinct().count()

In [5]:
# C. The number of distinct users who wrote reviews
output["n_user"] = review.map(lambda x: x["user_id"]).distinct().count()

In [6]:
# D. The top 10 users who wrote the largest numbers of reviews and the number of reviews they wrote
# top10_user
output["top10_user"] = review.map(lambda x: [x["user_id"],1]).reduceByKey(lambda x,y: x+y)\
                        .map(lambda x: list(x)).takeOrdered(10,key=lambda x: [-x[1],x[0]])
# output["top10_user"] = review.map(lambda x: [x["user_id"],1]).reduceByKey(lambda x,y: x+y)\
#                         .map(lambda x: list(x)).takeOrdered(10,key=lambda x: [-x[1],x[0]])

In [7]:
# E. The number of distinct businesses that have been reviewed
# n_business
output["n_business"] = review.map(lambda x: x["business_id"]).distinct().count()

In [3]:
# F. The top 10 businesses that had the largest numbers of reviews and the number of reviews they had
# top10_business
output["top10_business"] = review.map(lambda x: [x["business_id"],1]).reduceByKey(lambda x,y: x+y)\
                        .map(lambda x: list(x)).sortBy(lambda x:[-x[1],x[0]]).take(10)
# output["top10_business"] = review.map(lambda x: [x["business_id"],1]).reduceByKey(lambda x,y: x+y)\
#                         .map(lambda x: list(x)).takeOrdered(10,key=lambda x: [-x[1],x[0]])

In [9]:
with open(output_path,"w") as f:
    json.dump(output,f) 